In [1]:
# %load ../general_settings.py
import os
import shutil
import warnings
import subprocess
from time import strftime
from collections import defaultdict
from collections import namedtuple
from copy import copy
from functools import partial
from itertools import product
from itertools import combinations
from pathlib import Path

import numpy as np
import pandas as pd
import scipy.stats as stats
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
from colorama import Fore
from colorama import Style
from IPython.core.display import HTML

import joblib
import optuna

from scipy.cluster.hierarchy import linkage
from scipy.spatial.distance import squareform
from scipy.stats import gaussian_kde
from scipy.stats import probplot
from scipy.stats import yeojohnson
from sklearn.compose import make_column_transformer
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import Binarizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import StandardScaler
from lightgbm import LGBMClassifier
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.calibration import LinearSVC
from sklearn.ensemble import (
    BaggingClassifier,
    ExtraTreesClassifier,
    HistGradientBoostingClassifier,
    GradientBoostingClassifier,
)
from sklearn.feature_selection import RFECV, mutual_info_classif
from sklearn.kernel_approximation import Nystroem, RBFSampler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.neighbors import KNeighborsClassifier
from xgboost import XGBClassifier

ON_KAGGLE = os.getenv("KAGGLE_KERNEL_RUN_TYPE") is not None

# Colorama settings.
CLR = (Style.BRIGHT + Fore.BLACK) if ON_KAGGLE else (Style.BRIGHT + Fore.WHITE)
RED = Style.BRIGHT + Fore.RED
BLUE = Style.BRIGHT + Fore.BLUE
CYAN = Style.BRIGHT + Fore.CYAN
RESET = Style.RESET_ALL

FONT_COLOR = "#545454"
BACKGROUND_COLOR = "#F6F5F5"

CELL_HOVER = {  # for row hover use <tr> instead of <td>
    "selector": "td:hover",
    "props": "background-color: #F6F5F5",
}
TEXT_HIGHLIGHT = {
    "selector": "td",
    "props": "color: #545454; font-weight: bold",
}
INDEX_NAMES = {
    "selector": ".index_name",
    "props": "font-style: italic; background-color: #005D68; color: #F2F2F0;",
}
HEADERS = {
    "selector": "th:not(.index_name)",
    "props": "font-style: italic; background-color: #005D68; color: #F2F2F0;",
}
DF_STYLE = (INDEX_NAMES, HEADERS, TEXT_HIGHLIGHT)

# Utility functions.
def download_dataset_from_kaggle(user, dataset, directory):
    command = "kaggle datasets download -d "
    filepath = directory / (dataset + ".zip")
    if not filepath.is_file():
        subprocess.run((command + user + "/" + dataset).split())
        filepath.parent.mkdir(parents=True, exist_ok=True)
        shutil.unpack_archive(dataset + ".zip", "data")
        shutil.move(dataset + ".zip", "data")


def download_competition_from_kaggle(competition):
    command = "kaggle competitions download -c "
    filepath = Path("data/" + competition + ".zip")
    if not filepath.is_file():
        subprocess.run((command + competition).split())
        Path("data").mkdir(parents=True, exist_ok=True)
        shutil.unpack_archive(competition + ".zip", "data")
        shutil.move(competition + ".zip", "data")


# Html `code` block highlight. Must be included at the end of all imports!
HTML(
    """
<style>
code {
    background: rgba(42, 53, 125, 0.10) !important;
    border-radius: 4px !important;
}
</style>
"""
)


In [2]:
competition = "playground-series-s3e18"

if not ON_KAGGLE:
    download_competition_from_kaggle(competition)
    train_path = "data/train.csv"
    test_path = "data/test.csv"
else:
    train_path = f"/kaggle/input/{competition}/train.csv"
    test_path = f"/kaggle/input/{competition}/test.csv"

cols_to_skip = ["EC3", "EC4", "EC5", "EC6"]
train = pd.read_csv(train_path, index_col="id", usecols=lambda x: x not in cols_to_skip)
test = pd.read_csv(test_path, index_col="id")

orig = pd.read_csv("data/mixed_desc.csv")
orig["EC1_EC2_EC3_EC4_EC5_EC6".split("_")] = (
    orig["EC1_EC2_EC3_EC4_EC5_EC6"].str.split("_", expand=True).astype(np.int32)
)
orig = orig[train.columns]

continuous_variables = test.select_dtypes("float64").columns
discrete_variables = test.select_dtypes("int64").columns
targets = ["EC1", "EC2"]


In [3]:
train = pd.concat([train, orig], ignore_index=True)

# outlier_mask_train = (
#     (train.FpDensityMorgan1 >= test.FpDensityMorgan1.min())
#     & (train.FpDensityMorgan2 >= test.FpDensityMorgan2.min())
#     & (train.FpDensityMorgan3 >= test.FpDensityMorgan3.min())
#     & (train.VSA_EState9 >= test.VSA_EState9.min())
# )

outlier_mask_train = (
    (train.FpDensityMorgan1 >= test.FpDensityMorgan1.min())
    & (train.FpDensityMorgan2 >= test.FpDensityMorgan2.min())
    & (train.FpDensityMorgan3 >= test.FpDensityMorgan3.min())
    & (train.VSA_EState9 >= test.VSA_EState9.min())
    & (train.BertzCT <= test.BertzCT.max())
    & (train.Chi1 <= test.Chi1.max())
    & (train.Chi1n <= test.Chi1n.max())
    & (train.Chi1v <= test.Chi1v.max())
    & (train.Chi2n <= test.Chi2n.max())
    & (train.Chi3v <= test.Chi3v.max())
    & (train.Chi4n <= test.Chi4n.max())
    & (train.ExactMolWt <= test.ExactMolWt.max())
    & (train.HeavyAtomMolWt <= test.HeavyAtomMolWt.max())
    & (train.MinEStateIndex <= test.MinEStateIndex.max())
    & (train.PEOE_VSA6 <= test.PEOE_VSA6.max())
    & (train.SMR_VSA10 <= test.SMR_VSA10.max())
    & (train.SMR_VSA5 <= test.SMR_VSA5.max())
)

train = train[outlier_mask_train]
train = train.drop_duplicates(subset=np.r_[continuous_variables, discrete_variables])
train = train.sample(len(train), random_state=42)
train


,BertzCT,Chi1,Chi1n,Chi1v,Chi2n,Chi2v,Chi3v,Chi4n,EState_VSA1,EState_VSA2,...,PEOE_VSA7,PEOE_VSA8,SMR_VSA10,SMR_VSA5,SlogP_VSA3,VSA_EState9,fr_COO,fr_COO2,EC1,EC2
10890,236.164996,5.540111,3.481748,3.481748,2.481029,2.481029,1.308333,0.671209,42.723899,0.000000,...,0.000000,0.000000,0.000000,24.415866,0.000000,44.333333,0,0,1,1
1211,178.877124,5.609061,2.600764,2.600764,1.636538,1.636538,0.845375,0.348414,12.073272,12.170333,...,0.000000,6.420822,11.752550,12.524788,9.589074,33.250000,1,1,1,0
14311,301.783155,6.613392,4.023603,4.023603,2.725738,3.608976,1.861867,1.281644,0.000000,28.595232,...,17.696186,0.000000,6.076020,0.000000,0.000000,38.833333,0,0,1,1
10453,1024.310130,16.653503,9.868666,12.293774,7.316679,10.514180,7.796537,3.312893,101.198915,0.000000,...,6.923737,12.617665,21.552574,62.107304,36.756485,90.253387,0,0,1,1
12479,212.538082,4.914214,2.571070,3.971007,2.209975,3.110185,1.620551,0.368293,15.645394,6.606882,...,0.000000,0.000000,15.645394,0.000000,17.964475,21.506205,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5200,307.563775,6.123609,3.508653,3.508653,2.816709,2.816709,1.910460,1.030076,15.645394,6.606882,...,0.000000,0.000000,16.981741,6.103966,15.509617,37.213753,0,0,1,1
13439,405.799145,7.236382,4.299605,4.299605,4.573989,4.573989,2.571107,1.920817,36.992053,0.000000,...,0.000000,6.923737,5.969305,24.415866,9.589074,36.666667,1,1,1,1
5399,1333.668426,13.253691,9.668959,12.842059,8.182961,8.182961,7.371485,6.498959,94.271312,35.954542,...,0.000000,19.186948,0.000000,85.510795,22.701338,86.878753,0,0,1,1
861,278.545421,4.466326,2.973800,2.973800,2.002929,2.002929,0.985127,0.457018,0.000000,11.499024,...,12.132734,6.544756,0.000000,0.000000,0.000000,23.333333,0,0,1,1


In [4]:
numeric_descr = (
    train[continuous_variables]
    .describe(percentiles=[0.01, 0.05, 0.25, 0.50, 0.75, 0.95, 0.99])
    .drop("count")
    .T.rename(columns=str.title)
)

In [5]:
r2_scores = defaultdict(tuple)

for feature in continuous_variables:
    orig = train[feature].dropna()
    _, (*_, R_orig) = probplot(orig, rvalue=True)
    _, (*_, R_yeojohn) = probplot(yeojohnson(orig)[0], rvalue=True)

    if orig.min() >= 0:
        _, (*_, R_log) = probplot(np.log1p(orig), rvalue=True)
        _, (*_, R_sqrt) = probplot(np.sqrt(orig), rvalue=True)
    else:
        R_log, R_sqrt = np.nan, np.nan

    r2_scores[feature] = (
        R_orig * R_orig,
        R_yeojohn * R_yeojohn,
        R_log * R_log,
        R_sqrt * R_sqrt,
    )

r2_scores_frame = pd.DataFrame(r2_scores, index=("Original", "YeoJohnson", "Log", "Sqrt")).T

r2_scores_frame = (
    r2_scores_frame.assign(
        Winner=r2_scores_frame.idxmax(axis=1),
        m=r2_scores_frame.mean(axis=1),
    )
    .sort_values(by="m", ascending=False)
    .drop("m", axis=1)
)

yeojohnson_transform_cols = r2_scores_frame.query("Winner == 'YeoJohnson'").index
log_transform_cols = r2_scores_frame.query("Winner == 'Log'").index
sqrt_transform_cols = r2_scores_frame.query("Winner == 'Sqrt'").index

semi_constant_mask = np.isclose(numeric_descr["Min"], numeric_descr["50%"])
semi_constant_descr = numeric_descr[semi_constant_mask]
semi_const_cols_thresholds = semi_constant_descr["50%"].to_dict()

semi_const_cols = semi_const_cols_thresholds.keys()
yeojohnson_transform_cols = yeojohnson_transform_cols.drop(semi_const_cols, errors="ignore")
log_transform_cols = log_transform_cols.drop(semi_const_cols, errors="ignore")
sqrt_transform_cols = sqrt_transform_cols.drop(semi_const_cols, errors="ignore")


In [6]:
preliminary_preprocess = make_column_transformer(
    (
        make_pipeline(
            FunctionTransformer(func=np.log1p, feature_names_out="one-to-one"),
            StandardScaler(),
        ),
        log_transform_cols.to_list(),
    ),
    (
        make_pipeline(
            FunctionTransformer(func=np.sqrt, feature_names_out="one-to-one"),
            StandardScaler(),
        ),
        sqrt_transform_cols.to_list(),
    ),
    (
        PowerTransformer(method="yeo-johnson", standardize=True),
        yeojohnson_transform_cols.to_list(),
    ),
    (
        MinMaxScaler(),
        discrete_variables.to_list(),
    ),
    *[
        (
            Binarizer(threshold=thresh),
            [col],
        )
        for col, thresh in semi_const_cols_thresholds.items()
    ],
    remainder="drop",
    verbose_feature_names_out=False,
).set_output(transform="pandas")


In [7]:
pca_preprocess = make_column_transformer(
    (
        PCA(n_components=0.95, random_state=42),
        [
            "BertzCT",
            "ExactMolWt",
            "HeavyAtomMolWt",
            "Chi1",
            "Chi1n",
            "Chi1v",
            "Chi2n",
            "Chi2v",
            "Chi3v",
            "Chi4n",
        ],
    ),
    remainder="passthrough",
    verbose_feature_names_out=False,
).set_output(transform="pandas")


In [8]:
def drop_features(X):
    return X.drop(["fr_COO2"], axis=1)


dropper = FunctionTransformer(drop_features)


In [9]:
def logging_callback(study, frozen_trial):
    previous_best_value = study.user_attrs.get("previous_best_value", None)
    if previous_best_value != study.best_value:
        study.set_user_attr("previous_best_value", study.best_value)
        params = {key.split("__")[-1]: value for key, value in frozen_trial.params.items()}
        print(
            f"{CLR}Optuna Trial: {RED}{frozen_trial.number:03} {CLR}- ",
            f"{CLR}Best Value: {RED}{frozen_trial.value:.5f}\n",
            f"{CLR}Best Params: {RED}{params}",
            sep="",
        )


In [10]:
def define_model(trial, seed=None):
    selector_params = {
        "k": trial.suggest_int("selectkbest__k", 8, 20, step=2),
    }

    classifier_params = {
        "random_state": trial.suggest_categorical(
            "gradientboostingclassifier__random_state", [seed or 42]
        ),
        "max_features": trial.suggest_categorical(
            "gradientboostingclassifier__max_features", ["sqrt"]
        ),
        "learning_rate": trial.suggest_float(
            "gradientboostingclassifier__learning_rate", 1e-2, 1e-1
        ),
        "min_samples_leaf": trial.suggest_int(
            "gradientboostingclassifier__min_samples_leaf", 64, 256, step=16
        ),
    }

    # classifier_params = {
    #     "random_state": trial.suggest_categorical(
    #         "randomforestclassifier__random_state", [seed or 42]
    #     ),
    #     "min_samples_leaf": trial.suggest_int(
    #         "randomforestclassifier__min_samples_leaf", 16, 128, step=4
    #     ),
    #     "max_samples": trial.suggest_float(
    #         "randomforestclassifier__max_samples", 0.5, 1.0, step=0.1
    #     ),
    # }

    return make_pipeline(
        preliminary_preprocess,
        pca_preprocess,
        dropper,
        SelectKBest(**selector_params),
        GradientBoostingClassifier(**classifier_params),
    )


def objective(trial, X, y, seed=None):
    seed = seed or 42
    skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=seed)
    y_oof_proba = np.zeros_like(y, dtype=np.float32)
    model = define_model(trial, seed)

    for train_ids, valid_ids in skf.split(X, y):
        X_train, y_train = X.iloc[train_ids], y.iloc[train_ids]
        X_valid, y_valid = X.iloc[valid_ids], y.iloc[valid_ids]
        model.fit(X_train, y_train)
        y_oof_proba[valid_ids] = model.predict_proba(X_valid)[:, 1]
        # Pruning should be here if needed?

    return roc_auc_score(y, y_oof_proba)


def seed_study(seed, X, y, n_trials=100, n_jobs=-1):
    sampler = optuna.samplers.TPESampler(seed=seed)
    pruner = optuna.pruners.HyperbandPruner()  # Not used.
    study = optuna.create_study(direction="maximize", sampler=sampler, pruner=pruner)
    study.optimize(
        partial(objective, X=X, y=y, seed=seed),  # type: ignore
        n_trials=n_trials,
        callbacks=[logging_callback],
        n_jobs=n_jobs,
    )

    best_model = make_pipeline(
        preliminary_preprocess,
        pca_preprocess,
        dropper,
        SelectKBest(),
        GradientBoostingClassifier(),
    ).set_params(**study.best_params)

    best_value = np.round(study.best_value, 5)

    study_frame = study.trials_dataframe(
        attrs=("number", "value", "params", "state"),
    ).sort_values(by="value")

    return best_model, best_value, study_frame, study


In [11]:
np.random.seed(42)

models_ec1_path = Path(f"models/ec1/")
models_ec2_path = Path(f"models/ec2/")
models_ec1_path.mkdir(parents=True, exist_ok=True)
models_ec2_path.mkdir(parents=True, exist_ok=True)

n_seeds = 5
seeds = np.random.randint(0, 100, size=n_seeds).tolist()

X = train.drop(targets, axis=1)
y_ec1 = train.EC1
y_ec2 = train.EC2


In [ ]:
optuna.logging.set_verbosity(optuna.logging.ERROR)


def search_for_best_model(X, y, seeds, models_path, n_trials=100, n_jobs=-1):
    model_study = namedtuple("Study", ["best_model", "best_value", "study_frame", "study"])
    models = defaultdict(model_study)

    for seed in seeds:
        print(CLR + "Seed:", seed)
        best_model, best_value, study_frame, study = seed_study(
            seed, X, y, n_trials=n_trials, n_jobs=n_jobs
        )
        models[seed] = model_study(best_model, best_value, study_frame, study)
        joblib.dump(
            best_model,
            models_path
            / f"best_gb_seed_{seed:03}_{strftime('run_%Y_%m_%d_%H_%M_%S')}_value_{best_value:.5f}.pkl",
        )
        print()

    return models


print(CLR + "● EC1 Optimization:\n")
models_ec1 = search_for_best_model(X, y_ec1, seeds, models_ec1_path, 200)

print(CLR + "● EC2 Optimization:\n")
models_ec2 = search_for_best_model(X, y_ec2, seeds, models_ec2_path, 200)


● EC1 Optimization:

Seed: 51
Optuna Trial: 005 - Best Value: 0.70942
Best Params: {'k': 8, 'random_state': 51, 'max_features': 'sqrt', 'learning_rate': 0.07299607708391635, 'min_samples_leaf': 176}
Optuna Trial: 002 - Best Value: 0.71007
Best Params: {'k': 10, 'random_state': 51, 'max_features': 'sqrt', 'learning_rate': 0.09534026670539444, 'min_samples_leaf': 224}
Optuna Trial: 010 - Best Value: 0.71258
Best Params: {'k': 14, 'random_state': 51, 'max_features': 'sqrt', 'learning_rate': 0.06525971357257869, 'min_samples_leaf': 224}
Optuna Trial: 024 - Best Value: 0.71296
Best Params: {'k': 16, 'random_state': 51, 'max_features': 'sqrt', 'learning_rate': 0.08428937816764667, 'min_samples_leaf': 256}
Optuna Trial: 048 - Best Value: 0.71336
Best Params: {'k': 14, 'random_state': 51, 'max_features': 'sqrt', 'learning_rate': 0.08991749154227932, 'min_samples_leaf': 224}
Optuna Trial: 098 - Best Value: 0.71337
Best Params: {'k': 18, 'random_state': 51, 'max_features': 'sqrt', 'learning_rate

In [ ]:
import glob

models_ec1_paths = glob.glob(str(models_ec1_path / "*"))
models_ec2_paths = glob.glob(str(models_ec2_path / "*"))

loaded_models_ec1 = defaultdict(object)
loaded_models_ec2 = defaultdict(object)

for fname in models_ec1_paths:
    loaded_models_ec1[fname] = joblib.load(fname)

for fname in models_ec2_paths:
    loaded_models_ec2[fname] = joblib.load(fname)


In [ ]:
def fit_and_predict_proba(X, y, models, X_test):
    y_test = np.zeros_like(X_test.index, dtype=np.float32)

    for best_model in models.values():
        best_model.fit(X, y)
        y_test += best_model.predict_proba(X_test)[:, 1]

    return y_test / len(models)


y_test_ec1 = fit_and_predict_proba(X, y_ec1, loaded_models_ec1, test)
y_test_ec2 = fit_and_predict_proba(X, y_ec2, loaded_models_ec2, test)


In [ ]:
submission = pd.DataFrame(
    {
        "Id": test.index,
        "EC1": y_test_ec1,
        "EC2": y_test_ec2,
    }
).set_index("Id")

submission.to_csv("submission.csv")
submission.head().style.set_table_styles(DF_STYLE)

,EC1,EC2
Id,,
14838,0.421698,0.778844
14839,0.824086,0.823983
14840,0.778216,0.745381
14841,0.721107,0.796958
14842,0.808358,0.819328


In [ ]:
model = make_pipeline(
    preliminary_preprocess,
    pca_preprocess,
    dropper,
    SelectKBest(k=16),
    # CalibratedClassifierCV(LGBMClassifier(random_state=42, max_depth=2)),
    # RandomForestClassifier(random_state=42, min_samples_leaf=92,)
    GradientBoostingClassifier(
        random_state=42,
        
        max_features="sqrt",
        min_samples_leaf=512,
        learning_rate=0.1
    ),
)

scores_ec1 = cross_val_score(
    model,
    X,
    y_ec1,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring="roc_auc",
    n_jobs=5,
)

scores_ec2 = cross_val_score(
    model,
    X,
    y_ec2,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring="roc_auc",
    n_jobs=5,
)

scores_ec1.mean(), scores_ec2.mean()
# (0.7101198089948693, 0.5952289528513477)


(0.7133945059487774, 0.58873160856277)